# 0.라이브러리 import

In [6]:
import requests

# 1. 인덱스 만들기

In [8]:
# endpoint
#"{}/computervision/retrieval/indexes/[만들고자하는 인덱스 이름]?api-version=2023-05-01-preview".format()
index_name = "news"
endpoint_index = "{}/computervision/retrieval/indexes/{}?api-version=2023-05-01-preview".format(BASE_ENDPOINT, index_name)

# method:PUT

# headers
headers = {
    "Ocp-Apim-Subscription-Key" : API_KEY,
    "Content-Type" : "application/json"

}

# body
body = {
    'metadataSchema': {
    'fields': [
            {
        'name': 'cameraId',
        'searchable': False,
        'filterable': True,
        'type': 'string'
            },
            {
        'name': 'timestamp',
        'searchable': False,
        'filterable': True,
        'type': 'datetime'
            }
        ]
    },
    'features': [
        {
        'name': 'vision',
        'domain': 'surveillance'
        },
        {
        'name': 'speech'
        }
    ]
}

# status code: 201
response  = requests.put(endpoint_index, headers=headers, json=body)
print(response.status_code, response.content)

409 b'{"error":{"code":"AlreadyExists","message":"Index with the given name already exist."}}'


# 2. 인덱스에 비디오 파일 추가

In [9]:
ingestion_id = "news-cv" 
endpoint_video = "{}computervision/retrieval/indexes/{}/ingestions/{}?api-version=2023-05-01-preview".format(BASE_ENDPOINT, index_name, ingestion_id)

body2 = {
  'videos': [
    {
      'mode': 'add',
      'documentId': '02a504c9cd28296a8b74394ed7488045',
      'documentUrl': 'https://6b022storage.blob.core.windows.net/video-data/%E1%84%87%E1%85%B3%E1%84%85%E1%85%A1%E1%84%8C%E1%85%B5%E1%86%AF%20%E1%84%89%E1%85%A9%E1%84%83%E1%85%A9%E1%84%89%E1%85%B5%E1%84%8B%E1%85%A6%E1%84%89%E1%85%A5%20%E1%84%80%E1%85%A5%E1%84%83%E1%85%A2%20%E1%84%89%E1%85%B5%E1%86%BC%E1%84%8F%E1%85%B3%E1%84%92%E1%85%A9%E1%86%AF%E1%84%8B%E1%85%B5%20%E1%84%87%E1%85%A1%E1%86%AF%E1%84%89%E1%85%A2%E1%86%BC%E1%84%92%E1%85%A2%201%20200%E1%84%8B%E1%85%A7%20%E1%84%86%E1%85%A7%E1%86%BC%E1%84%8B%E1%85%B4%20%E1%84%8C%E1%85%AE%E1%84%86%E1%85%B5%E1%86%AB%E1%84%8B%E1%85%B5%20%E1%84%8C%E1%85%B5%E1%86%B8%E1%84%8B%E1%85%B3%E1%86%AF%20%E1%84%8B%E1%85%B5%E1%86%B6%E1%84%8B%E1%85%B3%E1%86%AF%20%E1%84%8B%E1%85%B1%E1%84%80%E1%85%B5%E1%84%8B%E1%85%A6%20%E1%84%8E%E1%85%A5%E1%84%92%E1%85%A2%E1%86%BB%E1%84%89%E1%85%B3%E1%86%B8%E1%84%82%E1%85%B5%E1%84%83%E1%85%A1.%20%E1%84%89%E1%85%B5%E1%86%BC%E1%84%8F%E1%85%B3%E1%84%92%E1%85%A9%E1%86%AF%20%E1%84%87%E1%85%B3%E1%84%85%E1%85%A1%E1%84%8C%E1%85%B5%E1%86%AF%20%E1%84%80%E1%85%A5%E1%84%83%E1%85%A2%E1%84%89%E1%85%B5%E1%86%BC%E1%84%8F%E1%85%B3%E1%84%92%E1%85%A9%E1%86%AF%20%E1%84%8C%E1%85%AE%E1%84%86%E1%85%B5%E1%86%AB...%20TikTok.mp4?sp=r&st=2025-03-25T08:16:32Z&se=2025-03-25T16:16:32Z&spr=https&sv=2024-11-04&sr=b&sig=%2FkXwvsiOkakis59xArTxC2kpJUuurBNRoMUYPFIqXJU%3D',
      'cameraId': 'camera1',
      'timestamp': '2025-03-24 17:40:33'
      }
  ]
}

response  = requests.put(endpoint_video, headers=headers, json=body2)
print(response.status_code, response.content)

409 b'{"error":{"code":"AlreadyExists","message":"Ingestion with the given name already exist."}}'


# 3. 상태 체크

In [10]:
endpoint_state = "{}computervision/retrieval/indexes/{}/ingestions?api-version=2023-05-01-preview&$top=20".format(BASE_ENDPOINT, index_name)

response = requests.get(endpoint_state, headers=headers)

print(response.status_code, response.text)
response_json = response.json()
print('STATUS: ', response_json["value"][0]["name"], response_json["value"][0]["state"])

200 {"value":[{"name":"news-cv","state":"Completed","batchName":"35232362-e488-4db2-9c63-f3ca0b2c9770","createdDateTime":"2025-03-25T08:45:24.9584428Z","lastModifiedDateTime":"2025-03-25T08:46:04.0364563Z"}]}
STATUS:  news-cv Completed


# 4. 조회

In [11]:
endpoint_search = "{}computervision/retrieval/indexes/{}:queryByText?api-version=2023-05-01-preview".format(BASE_ENDPOINT, index_name)

body3 = {
  'queryText': 'a man with black hoodie',
  'filters': {
    'stringFilters': [
      {
        'fieldName': 'cameraId',
        'values': [
          'camera1'
        ]
      }
    ],
    'featureFilters': ['vision']
  }
}

response = requests.post(endpoint_search, headers=headers, json=body3)
print(response.status_code, response.text)


200 {"value":[]}
